In [ ]:
!pip install numpy scikit-learn

Импортируем необходимые библиотеки и создадим такой набор данных, чтобы модель обучалась достаточно времени

In [1]:
import aiohttp
import asyncio
import time

import numpy as np
import requests

BASE_URL = "http://localhost:8000"
X = np.random.rand(4000000, 10).tolist()
y = np.random.rand(4000000).tolist()

Функция для отправки HTTP POST-запроса на обучение модели.

In [2]:
async def train_model(flag,session, model_name, model_type, X, y, config=None):
    payload = {
        "model_name": model_name,
        "model_type": model_type,
        "X": X,
        "y": y,
        "config": {"params": config} if config else {"params": {}},
    }
    if flag:
        async with session.post(f"{BASE_URL}/fit", json=payload) as response:
            await response.json()  # Возвращает JSON-ответ
    else: 
        requests.post(f"{BASE_URL}/fit", json=payload)

Функция для последовательного запуска

In [3]:
async def sequential_training():
    async with aiohttp.ClientSession() as session:
        started = time.time()
        
        await train_model(False, session, "test_model1", "LinearRegression", X, y, config={})
        
        await train_model(False, session, "test_model2", "LinearRegression", X, y, config={})
        
        print(f"{time.time() - started:.2f}")

Функция для параллельного запуска

In [4]:
async def parallel_training():
    async with aiohttp.ClientSession() as session:
        tasks = [
            train_model(True, session, "test_model3", "LinearRegression", X, y, config={}),
            train_model(True, session, "test_model4", "LinearRegression", X, y, config={}),
        ]
        started = time.time()
        await asyncio.gather(*tasks)
        print(f"{time.time() - started:.2f}")

In [5]:
async def main():
    print("Sequential training:")
    await sequential_training()
    print("Parallel training:")
    await parallel_training()

if __name__ == "__main__":
    await main()

Sequential training:
232.31
Parallel training:
197.89
